## Cors Fochno Structure from Motion (SfM) workflow
----------------------

The following workflow demonstrates the SfM process using open-source libs on data recently collected over a SSSI nature reserve called Cors Fochno in Mid Wales. This assumes basic knowledge of python and GIS usage.

This dataset consists of 171 RGB images captured using the Micasense Red Edge platform over a woodland on the edge of the Cors Fochno SSSI site, near Borth, Ceredigion. As well as the photos, a csv log of the photo capture GPS is included with the columns **#F=N X Y Z**, where the **#F** is the file name and the remainder (**X Y Z**) the coordinates. 

The following simple example uses the pycmac modules, culminating in the use of the Malt dense matching algorithm to produce a DSM and ortho-photos. Micmac has two dense matching implementations (Malt and PIMs), each with various modes and parameters.

Dependencies:

The scripts available in the UAS_SFM github site. 

QGIS 3 offers a versatile platform for this purpose for the final datasets, so please ensure this is installed. Once installed, install the HCMGIS plugin to use various web-based base-layers for visualisation purposes.


Installation of pdal **may** require a conda environment, so remember to change kernel at the appropriate point.

Assumed knowledge:

Basic GIS, image processing, remote sensing and command line use

Remember to query function args open a new cell (the plus icon) and write the function with a question mark - i.e

feature_match?

Data

The data is temporarily found in this google drive folder until QinetiQ find somewhere to store it.

https://drive.google.com/file/d/1hchKEW7XZRtuY-JpL1fSAs1GFgNhrNb9/view?usp=sharing

The file required is Woods.zip - extract this somewhere appropriate and change dir to this folder.

This contains:

- a collection of 3-band tif images
- woods.csv (the main GPS log)
- sub.csv (a lens caslibration subset)



### Workflow for creating a manually-defined subset

When faced with 100+ images, creating a calibration subset is best practice to speed up the orientation process. The following will create a manually defined subset using spatial data provided. It is better to define a subset by spatial properties than manually sifting through raw images. 


Import the UAV GPS ```woods.csv``` into QGIS using the delimited text file option, choose the appropriate delimiter and columns (space or tab). QGIS may look slightly different depending on whether you are using GNU/Linux, Mac or Windows but the functionality is the same. Use the selection tool to demarcate GPS points to create a subset. 

To select a subset, click on the select button and draw a polygon around the desired points which will turn yellow (see top left in the pic below). The location of each photo capture will be displayed as a point. If you have QGIS 3, for a google satellite background here install the HCMGIS plugin, otherwise the open-layers and Google earth view can be used in conjuction below.

<img src="Figures/QGIS.png" style="height:500px">

For a base map, install the open web map or open layers plugins (displayed is an open street map base map). Untick the base map layer and click on GEarthView (the icon below will be somewhere on your QGIS toolbar).

<img src="https://plugins.qgis.org/static/cache/7a/2b/7a2bc2eb1ff859db7f06845384276481.png" style="height:30px">

We can see by cross comparing with the QGIS selection is over an area of good topographic variability, which is important for a good calibration initialisation. 

**Ideally, a set of oblique images around the corner of an object (a building or similar) is ideal for the calibration subset** and where this is possible it should be used. However, with a fixed wing and nadir only imagery and nature of this site we do not have this luxury. In this instance then, an area of **variable topography with well connected imagery** is the next best thing. This is in order to attempt to satisfy the following:

- images convergent on same part of the scene,to facilitate the computation of the external orientation
- a scene with sufficient depth variation, for accurate focal length estimation
- overlapping images with common features in different positions, to have an accurate estimation of distortion

Next, right click on the .csv in layer properties and select ‘Save As’. Enter the name of the.csv to save ensuring the format dialogue is set to csv. 

<img src="Figures/GEView.jpg" style="height:500px">

The default separator of comma is recommended to use. Once set, click OK to save the “calib.csv”. 
Now the calibration subset can be used in the SfM process. 


### Feature detection, matching and orientation

First, the image coordinates are used to establish image pairs likely to overlap in order to detect features common to the overlapping images, which are later used to estimate the image poses in 3D space.


The following command performs the calibration, relative orientation and GPS-aided bundle adjustment. This may take a little while to run. 

Note the use of...

```bash 
-s calb.csv
```  
... the subset for the initial calibration. 


The Fraser lens distortion model is used and is the most sophisticated available with MicMac and generally returns the lowest RMSE.


In [6]:
Orientation.sh -e tif -u "30 +north" -c Fraser -t "woods.csv" -s "calib.csv" 

An output text may be seen documentating the iterative refinement of camera poses until completion.
Upon inspection of the “FraserGPSlog.txt” file in the working directory, we will see this near the bottom of the text.

```bash
| |  Residual = 0.429504 ;; Evol, Moy=2.74582e-07 ,Max=3.55555e-06
| |  Worst, Res 0.717166 for IMG_1305_RGB.tif,  Perc 98.1356 for IMG_1333_RGB.tif
| |  Cond , Aver 6.75509 Max 59.0378 Prop>100 0
CENTRE MEDIAN = [-0.290686,0.0135017,0.0115896]
--- End Iter 8 STEP 0
```
The text above  is the RMSE for the bundle adjustment ( ```Residual = 0.429504```), which, in this case is approximately 0.4 pixels.


**Alternatively it is possible to omit the calibration subset and allow MicMac's automated intialisation to do this for you by a quick orientation without radial distortion etc. which is then fed to the global orientation.**

```bash
Orientation.sh -e tif -u "30 +north" -c Fraser -t "woods.csv" 
```

In [ ]:
Orientation.sh -e tif -u "30 +north" -c Fraser -t "woods.csv" 

### Dense matching and outputs

Having completed the bundle adjustment phase, we can execute the dense matching algorithm. We will use the Malt-based algorithm in Ortho mode as this is generally most effective on mixed terrain and for producing high quality ortho-photos. 


**This dense matching can be executed using the bash script dense_cloud.sh with the following**

```bash
dense_cloud.sh -e tif -a Ortho -m Malt -u "30 +north" -i 1
```

In [ ]:
dense_cloud.sh -e tif -a Ortho -m Malt -u "30 +north" -i 0

### Mosaicking

After processing the working directory a Malt directory with an OUTPUT directory containing the DSM and orthomosaic, which will have been masked to include only valid pixels. 

There is no hard/fast rule for ortho-mosaicking, and experimentation may often be required. 

The default execution with the tawny function, will output a mosaic which will attempt to radiometrically equalise the mosaic.

Look at the first result in QGIS, which should like the one below!


As can be seen below there are seamlines (a patchwork effect) between the individual ortho-photos that make up the mosaic, due to uneven illumination conditions.

<img src="Figures/NoEgal.png" style="height:500px">

Hence, some radiometric equalisation will be required.

In [ ]:
mm3d Tawny Ortho-MEC-Malt RadiomEgal=1 Out=MoasaicNoEgal.tif;

# admin to get it to the output folder
cp Ortho-MEC-Malt/MoasaicNoEgal.tif OUTPUT/MoasaicNoEgal.tif;
cp Ortho-MEC-Malt/Orthophotomosaic.tfw OUTPUT/MoasaicNoEgal.tfw;


As can be seen below, the seamlines have disappeared but there now appears to be a radiometric drift in the mosaic with light and dark areas in a pattern. 

<img src="Figures/Egal1.png" style="height:500px">

The imagery trends from dark to light across the image and is not satisfactory as an output, hence further experimentation is required.

With default radiometric equalisation a radiometric drift is apparent in the ortho-mosaic, hence further experimentation is required.
The parameter
```python
DegRap=4
```
Fixes the degree of the global polynomial used in the radiometric equalisation and may reduce the drift.

In [ ]:
mm3d Tawny Ortho-MEC-Malt DegRap=4 Out=MoasaicRap4.tif;

# admin to get it to the output folder
cp Ortho-MEC-Malt/MoasaicNoEgal.tif OUTPUT/MoasaicRap4.tif;
cp Ortho-MEC-Malt/Orthophotomosaic.tfw OUTPUT/MoasaicRap4.tif;


As can be seem below, this parameter change appears to have had the desired effect and illumination is now near uniform (excluding the water where there is no 'real' incidence).

<img src="Figures/DegRap4.png" style="height:500px">

There are seamlines visible some of the water areas, but this is not of concern as this is rarely the main of object of interest with UAV imagery particulalry as water by its nature is not useful for photogrammetry.

A further option is to use feather mosaicking which occasionally may prove to be the best option, though there may be a loss of clarity in places. 

In [ ]:
orthomosaic.sh -f Ortho-MEC-Malt -u '30 +north' -t ossimFeatherMosaic -o OUTPUT/feathermosaic.tif

Look closely at the imagery and compare to the previous result. It is likely the previous is the best compromise....

<img src="Figures/Featherossim.png" style="height:500px">
